## Understanding Chaining And Runnables

### Load the ENV file

In [25]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)
load = load_dotenv('../.env')

print(os.getenv("LANGSMITH_PROJECT"))

MyFirstLangchainAutomation


### Create LLM Object

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.runnables import chain
import re

# ChatOllama (LLM) is a RunnableInterface itself
llm_qwen = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:latest",
    temperature=0.6,
    max_tokens=200
)

# ChatOllama (LLM) is a RunnableInterface itself
llm_deepseek = ChatOllama(
    base_url="http://localhost:11434",
    model="deepseek-r1:8b",
    temperature=0.6,
    max_tokens=200
)

# Making remove_think_block as a Runnable in order to chain it
@chain
def remove_think_block(text) -> str:
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL | re.IGNORECASE)

### Chaining the Runnables

In [27]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ("system", "You are an LLM expert"), 
    ("user", "What is the advantage of {action} AI models on Cloud ? /no_think")
    ])

# Chaining the prompt & LLM by providing the input of prompt to the LLM itself
chain = prompt | llm_qwen

chain.invoke({"action": "running"})

AIMessage(content='<think>\n\n</think>\n\nRunning AI models on the **cloud** offers numerous advantages, especially when compared to running them on-premises or using local infrastructure. Here are the key benefits of running AI models on the **cloud**:\n\n---\n\n### 1. **Scalability**\n- **On-demand resources**: Cloud platforms allow you to scale compute, storage, and memory resources up or down as needed.\n- **Handling large workloads**: AI models, especially those involving deep learning, require significant computational power, which the cloud can provide dynamically.\n\n---\n\n### 2. **Cost Efficiency**\n- **Pay-as-you-go model**: You only pay for the resources you use, which reduces upfront capital expenditure.\n- **No need for expensive hardware**: Cloud providers offer GPU and TPU instances that are optimized for AI workloads, eliminating the need for expensive on-premises hardware.\n\n---\n\n### 3. **Access to Advanced AI Tools and Services**\n- **Pre-built AI/ML services**: C

### Output STRING Parsing

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate([
    ("system", "You are the {tool} expert"),
    ("user", "What is the advantage of {action} the {tool} {article} {var} ?"),
])

# Chaining the outputParser along with the prompt & llm
chain = prompt | llm_deepseek | StrOutputParser()

output = chain.invoke({"action": "running", "tool": "LLM", "article": "on", "var": "Local machine"})

print(output)

### Chaining Multiple Chains | RunnableSequence

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

firstPrompt = ChatPromptTemplate([
    ("system", "You are the {tool} expert"),
    ("user", "What is the advantage of {action} the {tool} {article} {var} ? /no_think"),
])

detailedResponseChain = firstPrompt | llm_qwen | StrOutputParser()

followingPrompt = ChatPromptTemplate.from_template("""
                                     Analyse the response & get me just the headings from the {response}.
                                     Response should be in bullet points. Please do not provide explanation, but only precise output.
                                     """)

chainWithHeading = {"response": detailedResponseChain} | followingPrompt | llm_deepseek | StrOutputParser()

output = chainWithHeading.invoke({"action": "running", "tool": "LLM", "article": "on", "var": "Local machine"})

print(output)


<think>
Okay, let's break this down. The user wants a list of headings extracted from the assistant's response about running LLMs locally versus cloud-based.

First, I'll look at the structure provided by the assistant. It starts with an introduction section, then lists eight advantages under "Running a Large Language Model (LLM) on a local machine..." Each advantage has bullet points and explanations.

Next is the "When to Consider Local Deployment?" subsection followed by criteria like data privacy needs. Then there's a parallel part for cloud deployment considerations. Finally, there's a summary table comparing aspects between local and cloud options.

I need to make sure I capture all main sections including:
- The introduction paragraph
- Each numbered advantage with its explanation
- The subheadings "When to Consider Local Deployment?" and related criteria
- Similarly structured parts for Cloud-Based LLMs
- The summary section which includes the table

The user probably wants the

### Running Chains in Parallel | RunnableParallel

In [40]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

firstPrompt = ChatPromptTemplate([
    ("system", "You are the {tool} expert"),
    ("user", "What is the advantage of {action} the {tool} {article} {var} ? /no_think"),
])

detailedResponseChain = firstPrompt | llm_qwen | StrOutputParser()

nextPrompt = ChatPromptTemplate.from_template("""
                                     What is the capital of India ?
                                     Please do not provide LLM thoughts but rather directly the precise output.
                                     """)

chainWithHeading = nextPrompt | llm_deepseek | StrOutputParser()

parallelChain = RunnableParallel(chain1 = detailedResponseChain, chain2 = chainWithHeading)

output = parallelChain.invoke({"action": "running", "tool": "LLM", "article": "on", "var": "Local machine"})

# Output time reduced, even though 2 inputs are passed
print(output['chain1'])
print("\n\n")
print(output['chain2'])

<think>

</think>

Running a Large Language Model (LLM) on a local machine offers several advantages, depending on the use case, environment, and requirements. Here are the key benefits:

---

### 1. **Privacy and Data Security**
- **Data Confidentiality**: Sensitive data (e.g., personal information, business secrets) remains on the local machine and is not transmitted to external servers.
- **Compliance**: Helps meet regulatory and compliance requirements (e.g., GDPR, HIPAA) by keeping data within the organization's network.

---

### 2. **Control Over the Environment**
- **Customization**: You have full control over the model’s training data, fine-tuning, and deployment environment.
- **Customization of Inference**: You can tailor the model for specific tasks (e.g., code generation, translation, or domain-specific tasks) without relying on cloud APIs.

---

### 3. **Offline Access**
- **No Internet Dependency**: The model can operate without an internet connection, making it ideal fo

### Runnable Lambda

In [59]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

firstPrompt = ChatPromptTemplate([
    ("system", "You are the {tool} expert"),
    ("user", "What is the advantage of {action} the {tool} {article} {var} ? /no_think"),
])

# Chain 1
firstResponseChain = firstPrompt | llm_qwen | StrOutputParser()

followingPrompt = ChatPromptTemplate.from_template("""
                                     Analyse the response & get me just the headings from the {response}.
                                     Response should be in bullet points. Without explanation, but just the headings.
                                     """)

# Creating the RunnableLamda
decide_llm = lambda response: llm_deepseek if len(str(response)) > 300 else llm_qwen

llm_selector = RunnableLambda(decide_llm)

#Chain 2
chainWithHeading = {"response": firstResponseChain} | followingPrompt | llm_selector | StrOutputParser() | remove_think_block

output = chainWithHeading.invoke({"action": "running", "tool": "LLM", "article": "on", "var": "Local machine"})

print(output)


Here are the headings from the provided text:

*   Privacy and Data Security
    *   Data stays local
    *   Compliance
*   Control Over Model Behavior
    *   Customization
    *   Avoiding bias
    *   Reduced dependency on external services
*   Performance and Latency
    *   Faster response times
    *   Offline access
*   Cost Efficiency (in some cases)
    *   Reduced cloud costs
    *   Avoiding API fees
*   Scalability and Flexibility
    *   Custom hardware optimization
    *   Deployment flexibility
*   Avoiding Vendor Lock-In
*   Use Cases Where Local Deployment is Ideal
*   Potential Challenges
    *   High initial costs
    *   Technical expertise required
    *   Model size challenge
*   Summary


### Using @chain Decorator

In [60]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain

firstPrompt = ChatPromptTemplate([
    ("system", "You are the {tool} expert"),
    ("user", "What is the advantage of {action} the {tool} {article} {var} ? /no_think"),
])

# Chain 1
firstResponseChain = firstPrompt | llm_qwen | StrOutputParser()

followingPrompt = ChatPromptTemplate.from_template("""
                                     Analyse the response & get me just the headings from the {response}.
                                     Response should be in bullet points.
                                     """)

# Using the @chain decorator to make `decide_llm` Runnable
@chain
def decide_llm(response):
    response_text = str(response)
    if len(response_text) > 300:
        return llm_deepseek
    return llm_qwen

#Chain 2
chainWithHeading = {"response": firstResponseChain} | followingPrompt | decide_llm | StrOutputParser() | remove_think_block

output = chainWithHeading.invoke({"action": "running", "tool": "LLM", "article": "on", "var": "Local machine"})

print(output)


Here are the main headings from your response:

- **Privacy and Data Security**
- **Control Over the Model**
- **Offline Access**
- **Performance and Latency**
- **Cost Efficiency (in some cases)**
- **Custom Deployment**
- **Avoid Vendor Lock-in**

Let me know if you need help extracting specific ones!
